In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
#import pandas_profiling as pp

In [2]:
df = pd.read_csv('C:/Users/Elizabeth/Downloads/kidney_disease.csv')
df.describe()
(df.isnull().sum()/df.shape[0] * 100.00).round(2)

id                 0.00
age                2.25
bp                 3.00
sg                11.75
al                11.50
su                12.25
rbc               38.00
pc                16.25
pcc                1.00
ba                 1.00
bgr               11.00
bu                 4.75
sc                 4.25
sod               21.75
pot               22.00
hemo              13.00
pcv               17.50
wc                26.25
rc                32.50
htn                0.50
dm                 0.50
cad                0.50
appet              0.25
pe                 0.25
ane                0.25
classification     0.00
dtype: float64

In [3]:
df.drop('id', axis= 1, inplace= True) #No needed it

In [4]:
df.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [5]:
df['pcv'] = pd.to_numeric(df['pcv'], errors='coerce')
df['wc'] = pd.to_numeric(df['wc'], errors='coerce')
df['rc'] = pd.to_numeric(df['rc'], errors='coerce')
cat_col=[col for col in df.columns if df[col].dtype=='object']
for col in cat_col:
    print('{} has {} values '.format(col,df[col].unique()))
    print('\n')

rbc has [nan 'normal' 'abnormal'] values 


pc has ['normal' 'abnormal' nan] values 


pcc has ['notpresent' 'present' nan] values 


ba has ['notpresent' 'present' nan] values 


htn has ['yes' 'no' nan] values 


dm has ['yes' 'no' ' yes' '\tno' '\tyes' nan] values 


cad has ['no' 'yes' '\tno' nan] values 


appet has ['good' 'poor' nan] values 


pe has ['no' 'yes' nan] values 


ane has ['no' 'yes' nan] values 


classification has ['ckd' 'ckd\t' 'notckd'] values 




In [6]:
df['dm'].replace(to_replace = {'\tno':'no','\tyes':'yes',' yes':'yes'},inplace=True)
df['cad']= df['cad'].replace(to_replace ='\tno', value='no')
df['classification']= df['classification'].replace(to_replace ='ckd\t', value = 'ckd')
df['classification']= df['classification'].replace(to_replace ='notckd', value = 'not ckd')

for col in cat_col:
    print('{} has {} values '.format(col,df[col].unique()))
    print('\n')

rbc has [nan 'normal' 'abnormal'] values 


pc has ['normal' 'abnormal' nan] values 


pcc has ['notpresent' 'present' nan] values 


ba has ['notpresent' 'present' nan] values 


htn has ['yes' 'no' nan] values 


dm has ['yes' 'no' nan] values 


cad has ['no' 'yes' nan] values 


appet has ['good' 'poor' nan] values 


pe has ['no' 'yes' nan] values 


ane has ['no' 'yes' nan] values 


classification has ['ckd' 'not ckd'] values 




In [7]:
from pandas import errors


#df['classification']= df['classification'].map({'ckd':0, 'not ckd':1})
#df['classification']= pd.to_numeric(df['classification'], errors='coerce')
cat_col = [col for col in df.columns if df[col].dtype == 'object']
for col in cat_col:
    print(f"{col} has {df[col].unique()} values\n")

rbc has [nan 'normal' 'abnormal'] values

pc has ['normal' 'abnormal' nan] values

pcc has ['notpresent' 'present' nan] values

ba has ['notpresent' 'present' nan] values

htn has ['yes' 'no' nan] values

dm has ['yes' 'no' nan] values

cad has ['no' 'yes' nan] values

appet has ['good' 'poor' nan] values

pe has ['no' 'yes' nan] values

ane has ['no' 'yes' nan] values

classification has ['ckd' 'not ckd'] values



In [8]:
cat_cols = [col for col in df.columns if df[col].dtype =='object']
num_cols = [col for col in df.columns if df[col].dtype !='object']
num_cols = num_cols[:-1]

In [9]:
df_1 = df
def imputation(feature):
    mean = df[feature].mean()
    df[feature] = df[feature].fillna(mean)

for col in num_cols:
    imputation(col)

In [10]:
def impute_mode(feature):
    mode = df[feature].mode()[0]
    df[feature] = df[feature].fillna(mode)

for col in cat_cols:
    impute_mode(col)

In [11]:
(df.isnull().sum()/df.shape[0]*100.00).round(2)

age                0.00
bp                 0.00
sg                 0.00
al                 0.00
su                 0.00
rbc                0.00
pc                 0.00
pcc                0.00
ba                 0.00
bgr                0.00
bu                 0.00
sc                 0.00
sod                0.00
pot                0.00
hemo               0.00
pcv                0.00
wc                 0.00
rc                32.75
htn                0.00
dm                 0.00
cad                0.00
appet              0.00
pe                 0.00
ane                0.00
classification     0.00
dtype: float64

In [12]:
imp = KNNImputer(n_neighbors=11)
imp.fit_transform(df_1[num_cols])

def impute_mode(feature):
    mode = df_1[feature].mode()[0]
    df_1[feature] = df_1[feature].fillna(mode)

for col in cat_cols:
    impute_mode(col)

In [13]:
(df_1.isnull().sum()/df_1.shape[0]*100.00).round(2)

age                0.00
bp                 0.00
sg                 0.00
al                 0.00
su                 0.00
rbc                0.00
pc                 0.00
pcc                0.00
ba                 0.00
bgr                0.00
bu                 0.00
sc                 0.00
sod                0.00
pot                0.00
hemo               0.00
pcv                0.00
wc                 0.00
rc                32.75
htn                0.00
dm                 0.00
cad                0.00
appet              0.00
pe                 0.00
ane                0.00
classification     0.00
dtype: float64

In [15]:
df.to_csv('C:/Users/Elizabeth/Desktop/Gustavo Becerra/TDP/KD.csv')